# EE 451: Communications Systems
## Lecture 3 - Fourier Analysis Essentials

### Learning Objectives

By the end of this notebook, you will be able to:

1. Apply Fourier series to decompose periodic signals into harmonic components
2. Compute Fourier transforms of common signals using standard transform pairs
3. Utilize key Fourier transform properties (linearity, time-shift, frequency-shift, scaling)
4. Explain time-frequency duality and its implications for signal design
5. Define and calculate signal bandwidth using 3-dB and null-to-null criteria
6. Generate and visualize frequency spectra using Python/NumPy

---

## Setup and Imports

First, let's import the necessary libraries for signal processing and visualization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq, fftshift
from scipy import signal

# Set plotting style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## Part 1: Fourier Series - Square Wave Example

Fourier series allows us to decompose periodic signals into harmonic components. Let's examine a square wave:

$$x(t) = \frac{4A}{\pi}\left[\sin(2\pi f_0 t) + \frac{1}{3}\sin(6\pi f_0 t) + \frac{1}{5}\sin(10\pi f_0 t) + ...\right]$$

Only odd harmonics are present, with amplitudes decreasing as 1/n.

In [ ]:
# Parameters
f0 = 5  # Fundamental frequency (Hz)
T0 = 1 / f0  # Period
A = 1  # Amplitude
t = np.linspace(0, 1, 2000)  # Time vector (1 second)

# Generate square wave using Fourier series (sum of harmonics)
def fourier_series_square(t, f0, A, num_harmonics):
    """Generate square wave using Fourier series approximation"""
    x = np.zeros_like(t)
    for n in range(1, num_harmonics + 1, 2):  # Only odd harmonics
        x += (1/n) * np.sin(2 * np.pi * n * f0 * t)
    return (4 * A / np.pi) * x

# Create plots with different numbers of harmonics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Fourier Series Approximation of Square Wave', fontsize=16)

harmonics_list = [1, 3, 9, 21]

for idx, num_harmonics in enumerate(harmonics_list):
    ax = axes[idx // 2, idx % 2]
    x_approx = fourier_series_square(t, f0, A, num_harmonics)
    
    ax.plot(t, x_approx, linewidth=2)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.set_title(f'{num_harmonics} Harmonic{"s" if num_harmonics > 1 else ""}')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(0, 0.6)
    ax.set_ylim(-1.5, 1.5)

plt.tight_layout()
plt.show()

print(f"As we add more harmonics, the approximation improves.")
print(f"Note the Gibbs phenomenon (overshoot) at discontinuities.")

## Part 2: Fourier Transform - Rectangular Pulse

The Fourier transform extends frequency analysis to aperiodic signals. For a rectangular pulse:

$$x(t) = A \cdot \text{rect}(t/\tau) \quad \Longleftrightarrow \quad X(f) = A\tau \cdot \text{sinc}(\pi f\tau)$$

where $\text{sinc}(x) = \sin(x)/x$

In [ ]:
# Create rectangular pulse
tau = 0.2  # Pulse duration (seconds)
A = 1  # Amplitude
t = np.linspace(-1, 1, 2000)
dt = t[1] - t[0]

# Generate rectangular pulse
x = np.where(np.abs(t) < tau/2, A, 0)

# Compute FFT
N = len(x)
X = fftshift(fft(x)) * dt  # Scale by dt for continuous approximation
f = fftshift(fftfreq(N, dt))

# Theoretical Fourier transform
def sinc(x):
    """Sinc function with proper handling of x=0"""
    return np.sinc(x / np.pi)  # NumPy's sinc is sin(pi*x)/(pi*x)

X_theory = A * tau * sinc(f * tau)

# Plot time domain and frequency domain
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time domain
axes[0, 0].plot(t, x, linewidth=2, color='blue')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].set_title(f'Rectangular Pulse (τ = {tau} s)')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=0, color='k', linewidth=0.5)

# Magnitude spectrum (computed)
axes[0, 1].plot(f, np.abs(X), linewidth=2, color='red', label='FFT')
axes[0, 1].plot(f, np.abs(X_theory), '--', linewidth=2, color='green', label='Theory')
axes[0, 1].set_xlabel('Frequency (Hz)')
axes[0, 1].set_ylabel('|X(f)|')
axes[0, 1].set_title('Magnitude Spectrum')
axes[0, 1].set_xlim(-15, 15)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()
axes[0, 1].axhline(y=0, color='k', linewidth=0.5)
axes[0, 1].axvline(x=0, color='k', linewidth=0.5)

# Phase spectrum
axes[1, 0].plot(f, np.angle(X), linewidth=2, color='purple')
axes[1, 0].set_xlabel('Frequency (Hz)')
axes[1, 0].set_ylabel('Phase (rad)')
axes[1, 0].set_title('Phase Spectrum')
axes[1, 0].set_xlim(-15, 15)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0, color='k', linewidth=0.5)
axes[1, 0].axvline(x=0, color='k', linewidth=0.5)

# Highlight null-to-null bandwidth
null_freq = 1 / tau
axes[1, 1].plot(f, np.abs(X), linewidth=2, color='red')
axes[1, 1].axvline(x=-null_freq, color='orange', linestyle='--', linewidth=2, label=f'First Nulls (±{null_freq:.1f} Hz)')
axes[1, 1].axvline(x=null_freq, color='orange', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Frequency (Hz)')
axes[1, 1].set_ylabel('|X(f)|')
axes[1, 1].set_title(f'Null-to-Null Bandwidth = {2*null_freq:.1f} Hz')
axes[1, 1].set_xlim(-15, 15)
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print(f"Pulse duration: τ = {tau} s")
print(f"Null-to-null bandwidth: BW = 2/τ = {2/tau:.1f} Hz")
print(f"Time-bandwidth product: τ × BW = {tau * (2/tau):.1f}")

## Part 3: Time-Frequency Duality

One of the most important properties of the Fourier transform is **time-frequency duality**:

- Compression in time leads to expansion in frequency
- Expansion in time leads to compression in frequency
- The time-bandwidth product remains approximately constant

Let's explore this by varying the pulse duration.

In [ ]:
# Test different pulse durations
tau_values = [0.05, 0.1, 0.2, 0.4]
colors = ['blue', 'green', 'red', 'purple']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for tau, color in zip(tau_values, colors):
    # Time domain
    t = np.linspace(-1, 1, 2000)
    dt = t[1] - t[0]
    x = np.where(np.abs(t) < tau/2, 1, 0)
    
    # Frequency domain
    X = fftshift(fft(x)) * dt
    f = fftshift(fftfreq(len(x), dt))
    
    # Plot
    axes[0].plot(t, x + tau_values.index(tau) * 1.2, linewidth=2, color=color, label=f'τ = {tau} s')
    axes[1].plot(f, np.abs(X), linewidth=2, color=color, label=f'τ = {tau} s')

axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude (offset for clarity)')
axes[0].set_title('Time Domain: Pulse Duration')
axes[0].set_xlim(-0.5, 0.5)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('|X(f)|')
axes[1].set_title('Frequency Domain: Bandwidth')
axes[1].set_xlim(-30, 30)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTime-Frequency Duality Demonstration:")
print("═" * 60)
print(f"{'Pulse Duration (τ)':<20} {'Null-to-Null BW':<20} {'Product':<20}")
print("─" * 60)
for tau in tau_values:
    bw = 2 / tau
    product = tau * bw
    print(f"{tau:<20.2f} {bw:<20.1f} {product:<20.1f}")
print("═" * 60)
print("\nObservation: Narrower pulse → Wider spectrum")
print("            Wider pulse → Narrower spectrum")
print("            Time-bandwidth product ≈ 2 (constant)")

## Part 4: Frequency Shifting (Modulation Theorem)

The **modulation theorem** states that multiplying a signal by a carrier shifts its spectrum:

$$x(t) \cos(2\pi f_c t) \quad \Longleftrightarrow \quad \frac{1}{2}[X(f - f_c) + X(f + f_c)]$$

This is the foundation of amplitude modulation (AM).

In [ ]:
# Create baseband pulse
tau = 0.2
f_c = 10  # Carrier frequency (Hz)
t = np.linspace(-1, 1, 2000)
dt = t[1] - t[0]

# Baseband signal
x_baseband = np.where(np.abs(t) < tau/2, 1, 0)

# Modulated signal
carrier = np.cos(2 * np.pi * f_c * t)
x_modulated = x_baseband * carrier

# Compute spectra
X_baseband = fftshift(fft(x_baseband)) * dt
X_modulated = fftshift(fft(x_modulated)) * dt
f = fftshift(fftfreq(len(t), dt))

# Plot
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Baseband signal - time
axes[0, 0].plot(t, x_baseband, linewidth=2, color='blue')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].set_title('Baseband Signal')
axes[0, 0].set_xlim(-0.5, 0.5)
axes[0, 0].grid(True, alpha=0.3)

# Baseband signal - frequency
axes[0, 1].plot(f, np.abs(X_baseband), linewidth=2, color='blue')
axes[0, 1].set_xlabel('Frequency (Hz)')
axes[0, 1].set_ylabel('|X(f)|')
axes[0, 1].set_title('Baseband Spectrum')
axes[0, 1].set_xlim(-20, 20)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axvline(x=0, color='k', linewidth=0.5)

# Modulated signal - time
axes[1, 0].plot(t, x_modulated, linewidth=1, color='red', label='Modulated')
axes[1, 0].plot(t, x_baseband, '--', linewidth=2, color='blue', alpha=0.5, label='Envelope')
axes[1, 0].plot(t, -x_baseband, '--', linewidth=2, color='blue', alpha=0.5)
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Amplitude')
axes[1, 0].set_title(f'Modulated Signal (fc = {f_c} Hz)')
axes[1, 0].set_xlim(-0.5, 0.5)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Modulated signal - frequency
axes[1, 1].plot(f, np.abs(X_modulated), linewidth=2, color='red')
axes[1, 1].axvline(x=-f_c, color='orange', linestyle='--', linewidth=2, label=f'Carrier (±{f_c} Hz)')
axes[1, 1].axvline(x=f_c, color='orange', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Frequency (Hz)')
axes[1, 1].set_ylabel('|X(f)|')
axes[1, 1].set_title('Modulated Spectrum (Double Sideband)')
axes[1, 1].set_xlim(-20, 20)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axvline(x=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()

print(f"Carrier frequency: fc = {f_c} Hz")
print(f"Baseband bandwidth: BW = {1/tau:.1f} Hz")
print(f"Modulated bandwidth: 2 × BW = {2/tau:.1f} Hz (Double Sideband)")
print(f"\nNote: Spectrum is shifted to ±fc, creating upper and lower sidebands.")

## Part 5: Bandwidth Definitions

Different bandwidth definitions are used in practice:

1. **Null-to-null bandwidth**: Frequency span between first nulls
2. **3-dB bandwidth**: Frequency range where power is at least half the maximum
3. **Fractional power bandwidth**: Contains specified percentage of signal energy (e.g., 99%)

In [ ]:
# Create signals with different characteristics
tau = 0.1
t = np.linspace(-1, 1, 4000)
dt = t[1] - t[0]

# Rectangular pulse
x_rect = np.where(np.abs(t) < tau/2, 1, 0)

# Gaussian pulse (naturally bandlimited)
sigma = tau / 4
x_gauss = np.exp(-t**2 / (2 * sigma**2))

# Raised cosine pulse
x_rc = np.where(np.abs(t) < tau/2, 0.5 * (1 + np.cos(2*np.pi*t/tau)), 0)

# Compute spectra
X_rect = fftshift(fft(x_rect)) * dt
X_gauss = fftshift(fft(x_gauss)) * dt
X_rc = fftshift(fft(x_rc)) * dt
f = fftshift(fftfreq(len(t), dt))

# Calculate power spectra (normalized)
P_rect = np.abs(X_rect)**2
P_gauss = np.abs(X_gauss)**2
P_rc = np.abs(X_rc)**2

P_rect /= np.max(P_rect)
P_gauss /= np.max(P_gauss)
P_rc /= np.max(P_rc)

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Time domain
axes[0].plot(t, x_rect, linewidth=2, label='Rectangular', alpha=0.7)
axes[0].plot(t, x_gauss, linewidth=2, label='Gaussian', alpha=0.7)
axes[0].plot(t, x_rc, linewidth=2, label='Raised Cosine', alpha=0.7)
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Pulse Shapes')
axes[0].set_xlim(-0.3, 0.3)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Frequency domain (power spectrum, dB scale)
axes[1].plot(f, 10*np.log10(P_rect + 1e-10), linewidth=2, label='Rectangular', alpha=0.7)
axes[1].plot(f, 10*np.log10(P_gauss + 1e-10), linewidth=2, label='Gaussian', alpha=0.7)
axes[1].plot(f, 10*np.log10(P_rc + 1e-10), linewidth=2, label='Raised Cosine', alpha=0.7)
axes[1].axhline(y=-3, color='red', linestyle='--', linewidth=1, label='3-dB line')
axes[1].set_xlabel('Frequency (Hz)')
axes[1].set_ylabel('Power (dB)')
axes[1].set_title('Power Spectral Density')
axes[1].set_xlim(-30, 30)
axes[1].set_ylim(-40, 5)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nBandwidth Comparison:")
print("═" * 70)
print("Rectangular pulse: Infinite bandwidth (sinc spectrum)")
print("                   Null-to-null BW = 2/τ = 20 Hz")
print("                   99% power BW ≈ 10 Hz")
print("\nGaussian pulse:    Naturally bandlimited (Gaussian spectrum)")
print("                   3-dB BW well-defined")
print("                   No spectral nulls")
print("\nRaised cosine:     Smoother than rectangular")
print("                   Faster spectral decay than sinc")
print("                   Used in digital communications")
print("═" * 70)

## Part 6: Practical Example - Signal Design Trade-offs

In communication systems, we must balance:
- **Time duration** (symbol rate, data rate)
- **Bandwidth** (spectrum efficiency)
- **Power** (transmission efficiency)

Let's explore how pulse shaping affects these trade-offs.

In [ ]:
# Simulate a communication scenario
bit_rate = 1000  # bits per second
T_bit = 1 / bit_rate  # bit duration

# Generate time vector
t = np.linspace(-5*T_bit, 5*T_bit, 5000)
dt = t[1] - t[0]

# Different pulse shapes (all with same duration T_bit)
def rectangular_pulse(t, T):
    return np.where(np.abs(t) < T/2, 1, 0)

def raised_cosine_pulse(t, T, beta=0.5):
    """Raised cosine pulse with roll-off factor beta"""
    pulse = np.zeros_like(t)
    mask = np.abs(t) < T/2
    pulse[mask] = 0.5 * (1 + np.cos(2*np.pi*t[mask]/T))
    return pulse

# Create pulses
p_rect = rectangular_pulse(t, T_bit)
p_rc = raised_cosine_pulse(t, T_bit, beta=0.5)

# Compute spectra
P_rect = fftshift(fft(p_rect)) * dt
P_rc = fftshift(fft(p_rc)) * dt
f = fftshift(fftfreq(len(t), dt))

# Plot
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time domain comparison
axes[0, 0].plot(t*1000, p_rect, linewidth=2, label='Rectangular', color='blue')
axes[0, 0].plot(t*1000, p_rc, linewidth=2, label='Raised Cosine (β=0.5)', color='red')
axes[0, 0].set_xlabel('Time (ms)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].set_title(f'Pulse Shapes (Bit Rate = {bit_rate} bps)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xlim(-2, 2)

# Frequency domain comparison
axes[0, 1].plot(f/1000, np.abs(P_rect), linewidth=2, label='Rectangular', color='blue')
axes[0, 1].plot(f/1000, np.abs(P_rc), linewidth=2, label='Raised Cosine (β=0.5)', color='red')
axes[0, 1].axvline(x=bit_rate/1000, color='green', linestyle='--', linewidth=1, label=f'Bit rate ({bit_rate} Hz)')
axes[0, 1].axvline(x=-bit_rate/1000, color='green', linestyle='--', linewidth=1)
axes[0, 1].set_xlabel('Frequency (kHz)')
axes[0, 1].set_ylabel('|P(f)|')
axes[0, 1].set_title('Magnitude Spectrum')
axes[0, 1].set_xlim(-3, 3)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Bandwidth vs roll-off factor
beta_values = np.linspace(0, 1, 11)
bandwidths = bit_rate * (1 + beta_values)

axes[1, 0].plot(beta_values, bandwidths/1000, 'o-', linewidth=2, markersize=8, color='purple')
axes[1, 0].set_xlabel('Roll-off Factor (β)')
axes[1, 0].set_ylabel('Bandwidth (kHz)')
axes[1, 0].set_title('Raised Cosine Bandwidth vs Roll-off')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=bit_rate/1000, color='green', linestyle='--', label='Nyquist BW')
axes[1, 0].legend()

# Spectral efficiency
spectral_efficiency = bit_rate / bandwidths
axes[1, 1].plot(beta_values, spectral_efficiency, 'o-', linewidth=2, markersize=8, color='orange')
axes[1, 1].set_xlabel('Roll-off Factor (β)')
axes[1, 1].set_ylabel('Spectral Efficiency (bits/s/Hz)')
axes[1, 1].set_title('Spectral Efficiency vs Roll-off')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nCommunication System Design Trade-offs:")
print("═" * 70)
print(f"Bit rate: {bit_rate} bps")
print(f"Bit duration: {T_bit*1000:.2f} ms")
print("\nRectangular Pulse:")
print(f"  - Null-to-null BW: {2*bit_rate} Hz")
print(f"  - Sharp transitions in time")
print(f"  - Slow spectral decay (sinc)")
print(f"  - Intersymbol interference (ISI) issues")
print("\nRaised Cosine Pulse (β=0.5):")
print(f"  - Bandwidth: {bit_rate * 1.5} Hz")
print(f"  - Smooth transitions")
print(f"  - Faster spectral decay")
print(f"  - Zero ISI at sampling instants")
print(f"  - Spectral efficiency: {bit_rate/(bit_rate*1.5):.2f} bits/s/Hz")
print("\nConclusion: Raised cosine provides better bandwidth efficiency")
print("            and eliminates ISI, at cost of slight bandwidth increase.")
print("═" * 70)

## Summary and Key Takeaways

In this notebook, we explored:

1. **Fourier Series**: Decomposes periodic signals into harmonic components (square wave example)

2. **Fourier Transform**: Extends to aperiodic signals, relating time and frequency domains

3. **Time-Frequency Duality**: 
   - Compression in time → Expansion in frequency
   - Time-bandwidth product ≈ constant
   - ΔT · ΔF ≥ constant (uncertainty principle)

4. **Modulation Theorem**: Multiplication in time → Frequency shift (foundation of AM)

5. **Bandwidth Definitions**:
   - Null-to-null bandwidth
   - 3-dB bandwidth
   - Fractional power bandwidth

6. **Practical Applications**: Pulse shaping for bandwidth efficiency and ISI reduction

### Key Formulas

- **Rectangular pulse**: $x(t) = \text{rect}(t/\tau) \Longleftrightarrow X(f) = \tau \cdot \text{sinc}(\pi f\tau)$

- **Modulation theorem**: $x(t)\cos(2\pi f_c t) \Longleftrightarrow \frac{1}{2}[X(f-f_c) + X(f+f_c)]$

- **Time scaling**: $x(at) \Longleftrightarrow \frac{1}{|a|}X(f/a)$

- **Null-to-null BW** (rectangular pulse): $BW = 2/\tau$

### Next Steps

- Experiment with different signal types and parameters
- Explore convolution theorem and filtering
- Study energy spectral density and Parseval's theorem
- Apply these concepts to real communication systems

---

## Practice Exercises

Try modifying the code above to:

1. Create a triangular pulse and compute its Fourier transform
2. Experiment with different carrier frequencies in the modulation example
3. Compare the bandwidth of different pulse shapes (rectangular, Gaussian, raised cosine)
4. Simulate a multi-carrier system (OFDM-like) with multiple frequency-shifted pulses
5. Verify Parseval's theorem: Energy in time = Energy in frequency